In [1]:
from insilicoICH.study import ICHStudy

/scratch/brandon.nelson/demos/pediatric_ich_cadt/.demo/lib/python3.13/site-packages/monai/utils/module.py:399: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  pkg = __import__(module)  # top level module


In [10]:
head_phantoms.NIHPD_Head??

Init signature:
head_phantoms.NIHPD_Head(
    phantom_dir,
    age: float,
    symmetric=False,
    shape=None,
    skull_seg_method='otsu',
    add_sutures=False,
    add_fractures=True,
)
Source:        
class NIHPD_Head(HeadPhantom):
    '''
    loads MR brain atlas of mean `age`, downloaded from
    <https://www.bic.mni.mcgill.ca/~vfonov/nihpd/obj1> and saved in `phantom_dir`

    :param phantom_dir: str, directory holding .nii files from
        https://www.bic.mni.mcgill.ca/~vfonov/nihpd/obj1
    :param age: float, mean age of the atlas phantom to load. Note the brain
        atlases are stored as age ranges, thus the mean age is the mid point
        of the range (upper+lower)/2
    :param symmetry: optional, the atlases are provided in their natural
        asymmetric or artificially generated symmetric state, default is
        asymmetric, see article for more details:
    1. Fonov V, Evans AC, Botteron K, Almli CR, McKinstry RC, Collins DL.
        Unbiased average age-approp

In [11]:
from insilicoICH.phantoms import head_phantoms
phantom = head_phantoms.UNC_Head(phantom_dir='/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom/', age=2.0)
phantom

FileNotFoundError: No such file or no access: '/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom/UNCInfant012Atlases-2022-10-21/infant-2yr.nii.gz'

In [3]:
from VITools import get_available_phantoms
phantoms = get_available_phantoms()
phantoms

{'0.0 yr UNC Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.UNC_Head'>, age=0.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom')),
 '1.0 yr UNC Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.UNC_Head'>, age=1.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom')),
 '2.0 yr UNC Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.UNC_Head'>, age=2.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom')),
 '6.5 yr NIHPD Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.NIHPD_Head'>, age=6.5, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/NIHPD_Head_Phantom')),
 '9.0 yr NIHPD Head': functools.partial(<class 'insilicoICH.phantoms.head_phantoms.NIHPD_Head'>, age=9.0, phantom_dir=PosixPath('/projects01/didsr-aiml/brandon.nelson/ped

In [4]:
phantom = phantoms['1.0 yr UNC Head']()
phantom

FileNotFoundError: No such file or no access: '/projects01/didsr-aiml/brandon.nelson/pedsilicoICH/phantoms/UNC_Head_phantom/UNCInfant012Atlases-2022-10-21/infant-1yr.nii.gz'

In [3]:
lesion_volume = dict(IPH=[0.2, 10], SDH=[0.2, 10], EDH=[0.2, 10], Fracture=[0, 1]) # placeholders for fracture, eventually put in length, thickness etc
lesion_attn = dict(IPH=[40, 80], SDH=[40, 80], EDH=[40, 80], Fracture=[0, 1]) # placeholders for fracture, eventually put in length, thickness etc

In [4]:
import pandas as pd
from VITools import available_scanners

print(available_scanners)
study_plan = pd.concat([
    ICHStudy().metadata,
    ICHStudy.generate_from_distributions(get_available_phantoms(),
                            lesion_volume=lesion_volume,
                            lesion_attenuation=lesion_attn,
                            fracture_length=[150, 250],
                            views=[1000],
                            scanner_model=['Siemens_DefinitionFlash', 'GE_Lightspeed64'],
                            study_count=100,
                            seed=88)])
study_plan

['GE_Lightspeed16', 'GE_Lightspeed64', 'Scanner_Default', 'Siemens_DefinitionFlash']


,case_id,phantom,scanner_model,kVp,mA,pitch,views,scan_coverage,recon_kernel,slice_thickness,...,age,mass_effect,fracture_length,add_augmentation,texture_contrast,texture_scale,complexity,smoothness,irregularity,eccentricity
0,case_0000,10.5 yr NIHPD Head,Siemens_DefinitionFlash,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,10.50,0.651095,170,True,NaN,NaN,NaN,NaN,NaN,NaN
1,case_0001,2.0 yr UNC Head,Siemens_DefinitionFlash,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,2.00,0.268125,0,True,0.943187,19.543979,3.0,0.210047,0.303019,0.644476
2,case_0002,38.0 yr MIDA Head,Siemens_DefinitionFlash,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,38.00,0.786339,0,True,NaN,NaN,NaN,NaN,NaN,NaN
3,case_0003,12.0 yr NIHPD Head,Siemens_DefinitionFlash,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,12.00,0.873192,177,True,NaN,NaN,NaN,NaN,NaN,NaN
4,case_0004,11.5 yr NIHPD Head,GE_Lightspeed64,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,11.50,0.567150,0,True,0.016260,59.335687,3.0,0.351184,0.187087,0.430373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,case_0095,2.0 yr UNC Head,GE_Lightspeed64,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,2.00,0.221930,0,True,1.368422,18.335114,3.0,0.129984,0.372792,0.518568
96,case_0096,15.75 yr NIHPD Head,Siemens_DefinitionFlash,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,15.75,0.142653,242,True,NaN,NaN,NaN,NaN,NaN,NaN
97,case_0097,10.5 yr NIHPD Head,GE_Lightspeed64,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,10.50,0.471080,194,True,NaN,NaN,NaN,NaN,NaN,NaN
98,case_0098,10.5 yr NIHPD Head,GE_Lightspeed64,120.0,300.0,0.0,1000.0,dynamic,soft,1,...,10.50,0.582400,199,True,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
study = ICHStudy(study_plan)
study

study plan saved to: /scratch/brandon.nelson/demos/pediatric_ich_cadt/study_plan.csv



Input metadata:

      case_id              phantom            scanner_model    kVp     mA  \
0   case_0000   10.5 yr NIHPD Head  Siemens_DefinitionFlash  120.0  300.0   
1   case_0001      2.0 yr UNC Head  Siemens_DefinitionFlash  120.0  300.0   
2   case_0002    38.0 yr MIDA Head  Siemens_DefinitionFlash  120.0  300.0   
3   case_0003   12.0 yr NIHPD Head  Siemens_DefinitionFlash  120.0  300.0   
4   case_0004   11.5 yr NIHPD Head          GE_Lightspeed64  120.0  300.0   
..        ...                  ...                      ...    ...    ...   
95  case_0095      2.0 yr UNC Head          GE_Lightspeed64  120.0  300.0   
96  case_0096  15.75 yr NIHPD Head  Siemens_DefinitionFlash  120.0  300.0   
97  case_0097   10.5 yr NIHPD Head          GE_Lightspeed64  120.0  300.0   
98  case_0098   10.5 yr NIHPD Head          GE_Lightspeed64  120.0  300.0   
99  case_0099      0.0 yr UNC Head  Siemens_DefinitionFlash  120.0  300.0   

    pitch   views scan_coverage recon_kernel  slice_thick

In [6]:
study.run_all(parallel=True)

Running 100 simulation conditions
See logs at /scratch/brandon.nelson/demos/pediatric_ich_cadt/results/logs/VIT-BATCH_10-28-2025_19-19
10317324


Scans completed in parallel:   0%|          | 0/46 [00:00<?, ?it/s]



Input metadata:

      case_id              phantom            scanner_model    kVp     mA  \
0   case_0000   10.5 yr NIHPD Head  Siemens_DefinitionFlash  120.0  300.0   
1   case_0001      2.0 yr UNC Head  Siemens_DefinitionFlash  120.0  300.0   
2   case_0002    38.0 yr MIDA Head  Siemens_DefinitionFlash  120.0  300.0   
3   case_0003   12.0 yr NIHPD Head  Siemens_DefinitionFlash  120.0  300.0   
4   case_0004   11.5 yr NIHPD Head          GE_Lightspeed64  120.0  300.0   
..        ...                  ...                      ...    ...    ...   
95  case_0095      2.0 yr UNC Head          GE_Lightspeed64  120.0  300.0   
96  case_0096  15.75 yr NIHPD Head  Siemens_DefinitionFlash  120.0  300.0   
97  case_0097   10.5 yr NIHPD Head          GE_Lightspeed64  120.0  300.0   
98  case_0098   10.5 yr NIHPD Head          GE_Lightspeed64  120.0  300.0   
99  case_0099      0.0 yr UNC Head  Siemens_DefinitionFlash  120.0  300.0   

    pitch   views scan_coverage recon_kernel  slice_thick

In [10]:
len(sorted(study.results.case_id.unique()))

54